In [0]:
# -*- coding: utf-8 -*-
import dataiku
import re
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
pays_prepared = dataiku.Dataset("2018_pays_prepared")
pays_prepared_df = pays_prepared.get_dataframe()
toutes_organisations = dataiku.Dataset("2018_toutes_organisations")
toutes_organisations_df = toutes_organisations.get_dataframe()


# initialisation du set de sortie
pays_et_orgs = dataiku.Dataset("2018_pays_et_orgs")
pays_et_orgs.write_schema([{
    "name": "name_org",
    "type": "string",
},{
    "name": "abbrev_org",
    "type": "string",
},
    {"name": "liste_pays",
     "type": "string"
    }])

#jointure entre les deux datasets sur les codes d'organisations
# liste avec toutes les orgs de toutes_organisations
liste_orgs_dispos = [une_org["abbrev"] for une_org in toutes_organisations.iter_rows()]
liste_orgs_dispos = sorted(liste_orgs_dispos)
liste_name_orgs_dispos = [une_org["name"] for une_org in toutes_organisations.iter_rows()]
liste_name_orgs_dispos = sorted(liste_name_orgs_dispos)


# création d'un dictionnaire avec en valeur la liste des orgs auquelles la clé (nom du pays) appartient
dico_pays_orgs = {}
for row in pays_prepared.iter_rows():
    dico_pays_orgs[row["name"]]=re.split(r'\\', row["memberships"])

# dictionnaire avec en clé l'org et en valuer un eliste des pays membres: attention,
# il manque deux ou trois orgs à cause d'un tiret dans leur nom qui n'est pas passé par Wikidata
orgs_avec_pays = {}
for org in liste_orgs_dispos:
    liste_pays = []
    for pays in dico_pays_orgs:
        for une_org in dico_pays_orgs.get(pays):
            if une_org == org:
                liste_pays.append(pays)
    orgs_avec_pays[org]=liste_pays

orgs_avec_pays = sorted(orgs_avec_pays.items(), key=lambda t: t[0])
pays_des_orgs = []
for org in orgs_avec_pays:
    pays_des_orgs.append(org[1])

tableau = pd.DataFrame({'abbrev_org': liste_orgs_dispos})

tableau['name_org'] = pd.Series(liste_name_orgs_dispos, index=tableau.index)
tableau['liste_pays'] = pd.Series(pays_des_orgs, index=tableau.index)

#writer = pays_et_orgs.get_writer()
pays_et_orgs.write_schema_from_dataframe(tableau)
#trier le dico avec clés orgs et valuer pays
# ajouter ces pays dans le tableau

"""
with pays_et_orgs.get_writer() as writer:
    for org in orgs_avec_pays:
        for row in toutes_organisations.iter_rows():
            if row["abbrev"] == org:
                writer.write_row_array(row["name"],org, liste_pays)"""